In [2]:
import sys, cv2
import numpy as np
import tracktor_revised as tr
import matplotlib.pyplot as plt
import pickle
import os

In [3]:
# Open video file.
video_file        = '../Data/SF_n2_t2_3113.mp4'
cap               = cv2.VideoCapture(video_file)
n_frames          = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps               = cap.get(cv2.CAP_PROP_FPS)
print(f'{n_frames} frames, {fps} frames per second.')

# Contour detection parameters.
block_size,offset = 15,13
min_area,max_area = 20,1000
n_pix             = 5

56214 frames, 30.0 frames per second.


In [3]:
# List of frames to analyze.
#frameID_list = range(5000,50000,50)
frameID_list = range(5000,n_frames,1)

# Data structure to hold only the pixels belonging to a fish and their brightness.
fish = []

#Uncomment line to make this cell run, commented to force notebook to use the .pkl file for speed. (This cell takes over an hour.)
#for i in frameID_list:
    
    # Load the frame.
    cap.set(cv2.CAP_PROP_POS_FRAMES, i-1)
    ret,frame = cap.read()
    
    # Detect the contours.
    contours  = tr.contour_detect(frame,min_area,max_area,block_size,offset,n_pix)
    
    # Make a mask with value j inside fish number j and -1 elsewhere. 
    mask      = 0*cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) - 1
    for j in range(len(contours)):
        cv2.drawContours(mask, contours, j, j, -1)
    
    # Make and save a list of each fish's pixels.
    pixels=[]
    subFish=[]
    for j in range(len(contours)):
        pixels=(np.nonzero(mask==j))
        n =len(pixels[0]) # Number of pixels in J
        subSubFish=[]
        for k in range(n):
            row=pixels[0][k]
            col=pixels[1][k]
            rgb=frame[row,col]
            grey=round(.3*rgb[0]+.59*rgb[1]+.11*rgb[2],2)
            subSubFish.append([row,col,grey]) #Pixel
        subFish.append(np.array(subSubFish)) #Fish
    fish.append(subFish) #Frame

with open('fish.pkl','wb') as fh:
     pickle.dump(fish,fh)

IndentationError: unexpected indent (<ipython-input-3-f31202655608>, line 11)

In [4]:
with open('fish.pkl','rb') as fh:
     fish=pickle.load(fh)

In [ ]:
bins=np.linspace(0,256,20)
plt.figure(figsize=(10,10))
colorList=['r','b']
xlabel=('')
ylabel=('')
for i in range(len(fish)):
    for j in range(len(fish[i])):
        histArray=np.histogram(fish[i][j][:,2],bins=bins)
        plt.plot(histArray[1][:-1],histArray[0],color=colorList[j])
plt.show()

KeyboardInterrupt: 

In [ ]:
#Creates an array of the distance, brightness sum and brightness difference for the frame of greyFish 

pairFrameData=[]
for i in range(len(fish)):
    for j in range(i):
        distance=np.sqrt((fish[i][0]-fish[j][0])**2+(fish[i][1]-fish[j][1])**2)
        brightness=fish[i][2]+fish[j][2]
        difference=np.absolute(fish[i][2]-fish[j][2])
        pairFrameData.append([distance,brightness,difference])
pairFrameData=np.array(pairFrameData).T
print(pairFrameData)

In [ ]:
#Creates a 2d histogram of the first frame of greyFish using sum then difference of brightnesses
histArray=np.histogram(pairFrameData[0])
plt.plot(histArray[1][:-1],histArray[0])
plt.show()

heightValuesSum,bin1Sum,bin2sum=np.histogram2d(pairFrameData[0],pairFrameData[1])
plt.hist2d(pairFrameData[0],pairFrameData[1])
plt.show()

heightValuesDifference,bin1Difference,bin2Difference=np.histogram2d(pairFrameData[0],pairFrameData[2])
plt.hist2d(pairFrameData[0],pairFrameData[2])
plt.show()

In [ ]:
#Creates an array of the distance difference[0], brightness sum[1], and brightness difference[2] for every frame of greyFish
bins=np.linspace(0,500,20)
histDifferenceData=[]
for k in range(len(greyFish)):
    pairsData=[]
    for i in range(greyFish[k].shape[1]):
        for j in range(i):
            distance=np.sqrt((greyFish[k][0][i]-greyFish[k][0][j])**2+(greyFish[k][1][i]-greyFish[k][1][j])**2)
            brightness=greyFish[k][2][i]+greyFish[k][2][j]
            difference=np.absolute(greyFish[k][2][i]-greyFish[k][2][j])
            pairsData.append([distance,brightness,difference])
    pairsData=np.array(pairsData).T
    histDifferenceData.append(np.histogram2d(pairsData[0],pairsData[2],bins=[bin1Difference,bin2Difference]))

In [ ]:
#Calculates the euclidan difference between the first frame of histDifferenceData and every other frame using: the distance in [0], the brightnes sums in [1], and the brightness diffrerence in [2]
twodDifferenceArray=[]
for i in range(len(histDifferenceData)):
    twodDifferenceSubArray=[]
    twodDifferenceSubArray=np.linalg.norm(histDifferenceData[i][0]-histDifferenceData[0][0])
    twodDifferenceArray.append(twodDifferenceSubArray)

In [ ]:
#Plots histogram of distance values
histArray=np.histogram(twodDifferenceArray)
plt.plot(histArray[1][:-1],histArray[0])
plt.show()

In [6]:
crossedList=[]
for i in range(len(fish)):
    if len(fish[i])==1:
        crossedList.append(i)
#Take a list of each strech where the fish don't cross so that we can check which fish is which after they cross by comparing it to either the first or the longest streach uncrossed
#See 1 on fily's paper
longestDistance=[]
distance=0
for i in range(len(crossedList)):
    if (crossedList[i]-crossedList[i-1])>distance:
        distance=(crossedList[i]-crossedList[i-1])
        longestDistance=[distance,i,crossedList[i],crossedList[i-1]]
print(longestDistance)

[2482, 628, 26589, 24107]


In [54]:
#finding the center of each fish
#2 on fily's paper
fishMean=[]
overlapBooleanCheck=False
errorList=[]
for i in range(len(fish)):
    for j in range(len(crossedList)):
        if i==j:
            overlapBooleanCheck=True
    try:
        if overlapBooleanCheck==True:
            fishMean.append([[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])],[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])]])
        else:
            fishMean.append([[np.mean(fish[i][0].T[0]),np.mean(fish[i][0].T[1])],[np.mean(fish[i][1].T[0]),np.mean(fish[i][1].T[1])]])
    except:
        errorList.append(i)
    overlapBooleanCheck=False

In [57]:
print(errorList)

[1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1825, 2707, 2708, 2709, 2785, 2786, 2787, 2788, 2789, 2790, 2791, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059, 3060, 3229, 3233, 3234, 3235, 3290, 3291, 3292, 3293, 3294, 3595, 3596, 3598, 3599, 3600, 3601, 3658, 3659, 3660, 3733, 3734, 3735, 3736, 3737, 3770, 3771, 3772, 3773, 3781, 3782, 3783, 3784, 3785, 3786, 3787, 3896, 3897, 3918, 3919, 4062, 4063, 4064, 4065, 4066, 4067, 4155, 4156, 4389, 4551, 4589, 4590, 5152, 5153, 5154, 6433, 6436, 8500, 8536, 8537, 8538, 8539, 8540, 8541, 8542, 8543, 8544, 8545, 8546, 8547, 8548, 8945, 8946, 8947, 9123, 9124, 9125, 9126, 9127, 9128, 9129, 9130, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9138, 9139, 9140, 9508, 9509, 9510, 9511, 9512, 9513, 9514, 9515, 9516, 9517, 9518, 9519, 9520, 9521, 9522, 9523, 9524, 9525, 9526, 9527, 9528, 9529, 9530, 9531, 9532, 9533, 9534, 9535, 9536, 9537, 9538, 9571, 9572, 9573, 9574, 9575, 9576, 9577, 9578, 9579, 9580, 958

In [56]:
print(fishMean[0])
print(np.mean(fish[0][0].T[0]))
print(np.mean(fish[0][1].T[0]))

[[263.74038461538464, 206.14423076923077], [263.74038461538464, 206.14423076923077]]
263.74038461538464
242.18032786885246


In [28]:
#create array of distances between two fishes to find switching
#[[i,0-i_1,0][i,1-i+1,1][i,0-i+1,1][i,1-i+1,0]]
fishQuad=[]
for i in range(len(fishMean)-1):
    fishQuad.append([fishMean[i+1][0]-fishMean[i][0],fishMean[i+1][1]-fishMean[i][1],fishMean[i+1][1]-fishMean[i][0],fishMean[i+1][0]-fishMean[i][1]])
print(fishQuad[0])

[-0.7367002156721583, -0.5999999999999659, -33.578974358974364, 32.24227414330224]
